# Home task: pandas 

## Question 1

- Load the energy data from the file [Energy Indicators.xls](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls).
It is a list of indicators of energy supply and renewable electricity production from the United Nations for the year 2013.


- It should be put into a DataFrame with the variable name of "energy"


- Make sure to exclude the footer and header information from the datafile.


- The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:<br>
`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`


- Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule).


- For all countries which have missing data (e.g. data with `...`) make sure this is reflected as `np.NaN` values.


- Rename the following list of countries (for use in later questions):
    - `Republic of Korea`: `South Korea`,
    - `United States of America`: `United States`,
    - `United Kingdom of Great Britain and Northern Ireland`: `United Kingdom`,
    - `China, Hong Kong Special Administrative Region`: `Hong Kong`


- There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:
    - `Bolivia (Plurinational State of)` should be `Bolivia`,
    - `Switzerland17` should be `Switzerland`.


- Next, load the GDP data from the file ["world_bank.csv"](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). 
It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"


- Make sure to skip the header, and rename the following list of countries:
    - `Korea, Rep.`: `South Korea`,
    - `Iran, Islamic Rep.`: `Iran`,
    - `Hong Kong SAR, China`: `Hong Kong`


- Finally, load the "Sciamgo Journal and Country Rank data for [Energy Engineering and Power Technology"](http://www.scimagojr.com/countryrank.php?category=2102). It ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame "ScimEn"


- Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).


- The index of this DataFrame should be the name of the country, and the columns should be<br>
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', 2012', '2013', '2014', '2015']`

Function "answer_one" should return the resulted DataFrame (20 columns and 15 entries)

In [1]:
import numpy as np
import pandas as pd

In [7]:
def answer_one():
    file_url = 'Energy Indicators.xls'
    gdp_url = 'world_bank.csv'
    scim_url = 'scimagojr.xlsx'
  
    # Завантаження Energy
    energy = pd.read_excel(file_url, skiprows=17, skipfooter=38, usecols="C:F",
                           names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])
    energy.replace("...", np.nan, inplace=True)
    energy['Energy Supply'] *= 1_000_000

    rename_dict = {
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    }
    energy['Country'] = energy['Country'].replace(rename_dict)
    energy['Country'] = energy['Country'].str.replace(r'\d+', '', regex=True)
    energy['Country'] = energy['Country'].str.replace(r"\s*\(.*\)", "", regex=True)

    # Завантаження GDP
    gdp = pd.read_csv(gdp_url, skiprows=4)
    gdp.rename(columns={"Country Name": "Country"}, inplace=True)
    gdp.replace({
        "Korea, Rep.": "South Korea",
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"
    }, inplace=True)
    years = list(map(str, range(2006, 2016)))
    gdp = gdp[['Country'] + years]

    # Завантаження ScimEn
    scimen = pd.read_excel(scim_url)
    scimen = scimen.drop(columns=['Region'], errors='ignore')

    # Об'єднання датасетів
    merged_df = pd.merge(scimen, energy, on="Country", how="outer")
    merged_df = pd.merge(merged_df, gdp, on="Country", how="outer")

    # Вибір топ-15 країн
    merged_df = merged_df[merged_df['Rank'] <= 15]
    merged_df.set_index('Country', inplace=True)

    return merged_df


In [8]:
df = answer_one()
print(df.shape)
df.head(15)


(15, 20)


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Australia,14.0,27993.0,27254.0,608021.0,84747.0,21.72,234.0,5386000000,231,11.810810,7.484176e+11,8.550075e+11,1.056112e+12,9.287621e+11,1.148838e+12,1.398611e+12,1.547532e+12,1.577123e+12,1.468265e+12,1.351296e+12
Brazil,15.0,27316.0,26888.0,307607.0,71778.0,11.26,154.0,12149000000,59,69.648030,1.107627e+12,1.397114e+12,1.695855e+12,1.666996e+12,2.208838e+12,2.616156e+12,2.465228e+12,2.472820e+12,2.456044e+12,1.802212e+12
Canada,8.0,41209.0,40390.0,915491.0,142691.0,22.22,284.0,10431000000,296,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
China,1.0,402188.0,400158.0,5077232.0,3511673.0,12.62,343.0,127191000000,93,19.754910,2.752119e+12,3.550328e+12,4.594337e+12,5.101691e+12,6.087192e+12,7.551546e+12,8.532185e+12,9.570471e+12,1.047562e+13,1.106157e+13
France,12.0,30810.0,30139.0,547122.0,89958.0,17.76,221.0,10597000000,166,17.020280,2.317862e+12,2.655817e+12,2.926803e+12,2.700076e+12,2.646230e+12,2.870409e+12,2.683007e+12,2.816078e+12,2.861236e+12,2.442483e+12
Germany,6.0,50906.0,49773.0,777362.0,160302.0,15.27,252.0,13261000000,165,17.901530,3.046309e+12,3.484057e+12,3.808786e+12,3.479801e+12,3.468154e+12,3.824829e+12,3.597897e+12,3.808086e+12,3.965801e+12,3.423568e+12
India,3.0,81853.0,79757.0,968523.0,369256.0,11.83,237.0,33195000000,26,14.969080,9.402599e+11,1.216736e+12,1.198895e+12,1.341888e+12,1.675616e+12,1.823052e+12,1.827638e+12,1.856722e+12,2.039126e+12,2.103588e+12
Iran,11.0,32080.0,31725.0,634135.0,177894.0,19.77,198.0,9172000000,119,5.707721,2.656022e+11,3.497366e+11,4.060709e+11,4.140591e+11,4.870696e+11,6.290823e+11,6.440193e+11,5.003998e+11,4.622848e+11,4.091917e+11
Italy,9.0,38700.0,36909.0,639473.0,147302.0,16.52,209.0,6530000000,109,33.667230,1.958564e+12,2.222524e+12,2.417508e+12,2.209484e+12,2.144936e+12,2.306974e+12,2.097929e+12,2.153226e+12,2.173256e+12,1.845428e+12


## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 2
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [26]:
def answer_two():
    Top15 = answer_one()
    
    avgGDP = Top15.loc[:, '2006':'2015'].mean(axis=1).sort_values(ascending=False)

    return avgGDP

In [27]:
print(answer_two())

Country
United States         1.572243e+13
China                 6.927707e+12
Japan                 5.239642e+12
Germany               3.590729e+12
United Kingdom        2.777505e+12
France                2.692000e+12
Italy                 2.152983e+12
Brazil                1.988889e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.406644e+12
South Korea           1.221328e+12
Australia             1.207997e+12
Iran                  4.567516e+11
dtype: float64


### Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [11]:
def answer_three():
    Top15 = answer_one()
    avgGDP = answer_two()
    
    sixth_country = avgGDP.index[5]

    gdp_change = Top15.loc[sixth_country, '2015'] - Top15.loc[sixth_country, '2006']

    return gdp_change


In [12]:
print(answer_three())


124621907951.68018


### Question 4

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [13]:
def answer_four():
    Top15 = answer_one()
    Top15['Citation Ratio'] = Top15['Self-citations'] / Top15['Citations']

    max_country = Top15['Citation Ratio'].idxmax()
    max_value = float(Top15.loc[max_country, 'Citation Ratio'])
    
    return (max_country, max_value)


In [14]:
print(answer_four())

('China', 0.6916510807463594)


### Question 5

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [15]:
def answer_five():
    Top15 = answer_one()
    
    Top15["Population Estimate"] = Top15["Energy Supply"] / Top15["Energy Supply per Capita"]
    Top15["Population Estimate"] = Top15["Population Estimate"].astype(float)
    
    third_most_populous = Top15["Population Estimate"].nlargest(3).idxmin()
    
    return third_most_populous


In [16]:
print(answer_five())

Brazil


### Question 6
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [17]:
def answer_six():
    Top15 = answer_one()
    
    Top15["Population Estimate"] = Top15["Energy Supply"] / Top15["Energy Supply per Capita"]
    Top15["Citable Documents per Capita"] = Top15["Citable documents"] / Top15["Population Estimate"]
    
    correlation = Top15["Citable Documents per Capita"].corr(Top15["Energy Supply per Capita"])
    
    return correlation


In [18]:
print(answer_six())

0.810917934653874


### Question 7
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [21]:
def answer_seven():
    Top15 = answer_one()
    Top15["Population Estimate"] = Top15["Energy Supply"] / Top15["Energy Supply per Capita"]

    ContinentDict = {'China':'Asia', 
                     'United States':'North America', 
                     'Japan':'Asia', 
                     'United Kingdom':'Europe', 
                     'Russian Federation':'Europe', 
                     'Canada':'North America', 
                     'Germany':'Europe', 
                     'India':'Asia',
                     'France':'Europe', 
                     'South Korea':'Asia', 
                     'Italy':'Europe', 
                     'Spain':'Europe', 
                     'Iran':'Asia',
                     'Australia':'Australia', 
                     'Brazil':'South America'}

    Top15['Continent'] = Top15.index.map(ContinentDict)

    grouped = Top15.groupby('Continent')['Population Estimate'].agg(
        size='size',
        sum='sum',
        mean='mean',
        std='std'
    )

    return grouped


In [22]:
print(answer_seven())

               size               sum              mean           std
Continent                                                            
Asia              5   2898666386.6106   579733277.32212  6.790979e+08
Australia         1   23316017.316017   23316017.316017           NaN
Europe            6  394058699.474436   78811739.894887  3.813228e+07
North America     2   35239864.864865   35239864.864865           NaN
South America     1  205915254.237288  205915254.237288           NaN
